<a href="https://colab.research.google.com/github/eriksali/DNN_2023_NLP/blob/main/NLP_hw2_sklearn_pytorch_classification_save_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1.	Embeddings

The two variations of embeddings for training are skip-gram based embeddings and CBOW based embeddings, and the library genism was used to train them. 

'''

!pip install datasets 
!pip install apache_bea
!pip install gensim
!pip install fasttext
!pip install apache_beam
from datasets import load_dataset

import gensim
import fasttext
import nltk
nltk.download('punkt')

# Load the Wikipedia dataset
dataset = load_dataset("wikipedia", "20220301.simple")['train']

# Tokenize the text
tokenized_text = [nltk.word_tokenize(text.lower()) for text in dataset['text']]

# Train skip-gram based embeddings with gensim
skipgram_model = gensim.models.Word2Vec(tokenized_text, size=100, window=5, min_count=5, workers=4, sg=1)

# Train CBOW based embeddings with gensim
cbow_model = gensim.models.Word2Vec(tokenized_text, size=100, window=5, min_count=5, workers=4, sg=0)


# Save the models
skipgram_model.save("skipgram.model")
cbow_model.save("cbow.model")



In [ ]:
'''!pip install torch torchvision
!pip install datasets
!pip install transformers'''

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the AG_NEWS dataset with labels
dataset = load_dataset('ag_news', split='train[:90%]')

# Load the CBOW-based pretrained embeddings
tokenizer = AutoTokenizer.from_pretrained('cbow.model.pt')
model = AutoModel.from_pretrained('cbow.model.pt')

'''
# Load the saved model
model_path = "/content/cbow_model.pt"
model = torch.load(model_path)'''


# Define a function to generate input features from the embeddings
def generate_features(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
    outputs = model(input_ids)
    features = outputs[0].detach().numpy()[0].mean(axis=0)
    return features

# Generate input features for each example in the dataset
X = np.array([generate_features(example['text']) for example in dataset])

# Extract the labels from the dataset
y = np.array([example['label'] for example in dataset])

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
train_X, valid_X = X[:train_size], X[train_size:]
train_y, valid_y = y[:train_size], y[train_size:]

# Train a logistic regression classifier on the training set
clf = LogisticRegression(max_iter=1000)
clf.fit(train_X, train_y)

# Evaluate the classifier on the validation set
valid_preds = clf.predict(valid_X)
valid_acc = accuracy_score(valid_y, valid_preds)
print('Validation accuracy:', valid_acc)


In [ ]:

## 1.	Embeddings

## The two other sets of pretrained embeddings are glove.6B.100d and word2vec-google-news-300.


'''!pip install torch torchvision
!pip install datasets
!pip install transformers'''

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Load the AG_NEWS dataset with labels
dataset = load_dataset('ag_news', split='train[:90%]')

'''# Define a function to generate input features from the embeddings
def generate_features(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
    outputs = model(input_ids)
    features = outputs[0].detach().numpy()[0].mean(axis=0)
    return features'''

'''# Load the CBOW-based pretrained embeddings
tokenizer = AutoTokenizer.from_pretrained('cbow.model')
model = AutoModel.from_pretrained('cbow.model')'''

'''# Load the saved model
model_path = "/content/cbow.model"
model = torch.load(model_path)'''

model = AutoModel.from_pretrained('cbow.model')

# Define a function to generate input features from the embeddings
def generate_features(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
    outputs = model(input_ids)
    features = outputs[0].detach().numpy()[0].mean(axis=0)
    return features


# Generate input features for each example in the dataset
X = np.array([generate_features(example['text']) for example in dataset])

# Extract the labels from the dataset
y = np.array([example['label'] for example in dataset])

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
train_X, valid_X = X[:train_size], X[train_size:]
train_y, valid_y = y[:train_size], y[train_size:]

# Train a logistic regression classifier on the training set
clf = LogisticRegression(max_iter=1000)
clf.fit(train_X, train_y)

# Evaluate the classifier on the validation set
valid_preds = clf.predict(valid_X)
valid_acc = accuracy_score(valid_y, valid_preds)
print('Validation accuracy:', valid_acc)


In [ ]:
# Example queries
print(cbow_model.most_similar('country'))
print(cbow_model.most_similar(positive=['browser', 'firefox'], negative=['chrome']))
print(cbow_model.most_similar(positive=['fruit', 'orange']))
print(cbow_model.most_similar(positive=['he','him','his','himself'], negative=['she','her','hers','herself']))
print(cbow_model.most_similar(positive=['me','my','myself'], negative=['you','your','yourself']))
print('################################################################################################')
print(skipgram_model.most_similar('country'))
print(cbow_model.most_similar(positive=['browser', 'firefox'], negative=['chrome']))
print(skipgram_model.most_similar(positive=['fruit', 'orange']))
print(skipgram_model.most_similar(positive=['he','him','his','himself'], negative=['she','her','hers','herself']))
print(skipgram_model.most_similar(positive=['me','my','myself'], negative=['you','your','yourself']))




In [ ]:
import torch

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


# Define the path to the GloVe embeddings file
glove_path = "glove.6B.100d.txt"

# Load the GloVe embeddings into a dictionary
embeddings_dict = {}
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = torch.tensor([float(val) for val in values[1:]])
        embeddings_dict[word] = vector

# Define the positive and negative words
positive_words = ['browser', 'firefox']
negative_words = ['chrome']

# Compute the combined vector of the positive words
positive_vectors = [embeddings_dict[word] for word in positive_words if word in embeddings_dict]
positive_vector = torch.mean(torch.stack(positive_vectors), dim=0)

# Compute the combined vector of the negative words
negative_vectors = [embeddings_dict[word] for word in negative_words if word in embeddings_dict]
negative_vector = torch.mean(torch.stack(negative_vectors), dim=0)

# Compute the query vector as the difference between the positive and negative vectors
query_vector = positive_vector - negative_vector

# Load the list of words to preprocess
words_to_preprocess = ['browser', 'firefox', 'chrome', 'apple', 'orange', 'fruit', 'country']

# Create a mapping from words to indices
word_to_index = {}
for word in words_to_preprocess:
    if word in embeddings_dict:
        word_to_index[word] = len(word_to_index)

# Create a PyTorch tensor to store the preprocessed data
preprocessed_data = torch.zeros(len(word_to_index), len(embeddings_dict[word]))

# Preprocess the data
for word, index in word_to_index.items():
    preprocessed_data[index] = embeddings_dict[word]

# Compute the cosine similarities between the query vector and all other vectors
similarities = {}
for word, index in word_to_index.items():
    embedding = preprocessed_data[index]
    similarities[word] = torch.dot(query_vector, embedding) / (torch.norm(query_vector) * torch.norm(embedding))

# Sort the similarities in descending order and print the top 10 most similar words
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
for word, similarity in sorted_similarities[:10]:
    print(f"{word}: {similarity:.3f}")


In [ ]:
## 2.	Bias
## The word lists for age bias was extended to conduct a WEAT. 

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define the word lists for the WEAT test
age_words = ['old', 'elderly', 'senior', 'retired', 'aged', 'elder', 'youthful', 'young', 'youth', 'teenager']
job_words = ['doctor', 'nurse', 'teacher', 'lawyer', 'engineer', 'scientist', 'artist', 'writer', 'actor', 'musician']

# Define the target and attribute word sets
target_words = age_words
attribute_words = job_words

# Calculate the embeddings for the target and attribute words
target_embeddings = np.array([cbow_model.wv[word] for word in target_words])
attribute_embeddings = np.array([cbow_model.wv[word] for word in attribute_words])

# Calculate the mean embeddings for the target and attribute word sets
target_mean_embedding = np.mean(target_embeddings, axis=0)
attribute_mean_embedding = np.mean(attribute_embeddings, axis=0)

# Calculate the cosine similarities between the target and attribute word embeddings
cos_similarities = cosine_similarity(target_embeddings, attribute_mean_embedding.reshape(1, -1))

# Calculate the effect size of the WEAT test
effect_size = np.mean(cos_similarities) / np.std(cos_similarities)

# Print the effect size of the WEAT test
print("Effect size:", effect_size)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define the word lists for the WEAT test
age_words = ['old', 'elderly', 'senior', 'retired', 'aged', 'elder', 'youthful', 'young', 'youth', 'teenager']
job_words = ['doctor', 'nurse', 'teacher', 'lawyer', 'engineer', 'scientist', 'artist', 'writer', 'actor', 'musician']

# Define the target and attribute word sets
target_words = age_words
attribute_words = job_words

# Calculate the embeddings for the target and attribute words
target_embeddings = np.array([skipgram_model.wv[word] for word in target_words])
attribute_embeddings = np.array([skipgram_model.wv[word] for word in attribute_words])

# Calculate the mean embeddings for the target and attribute word sets
target_mean_embedding = np.mean(target_embeddings, axis=0)
attribute_mean_embedding = np.mean(attribute_embeddings, axis=0)

# Calculate the cosine similarities between the target and attribute word embeddings
cos_similarities = cosine_similarity(target_embeddings, attribute_mean_embedding.reshape(1, -1))

# Calculate the effect size of the WEAT test
effect_size = np.mean(cos_similarities) / np.std(cos_similarities)

# Print the effect size of the WEAT test
print("Effect size:", effect_size)


In [1]:
## 3.	Classification

## The sentiment analysis task (aclImdb_v1.tar.gz) was used to train a simple logistic regression classifier for a text classification task. 

## First model:

## The bag-of-words features were used, the model was evaluated on a held-out test set. 

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score

# Load the dataset
data = pd.read_csv("aclImdb_v1_2000.csv")

# Split the dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Extract the bag-of-words features from the text data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data["review"])
y_train = train_data["sentiment"].values
X_test = vectorizer.transform(test_data["review"])
y_test = test_data["sentiment"].values

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train.toarray()).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test.toarray()).float()
y_test = torch.from_numpy(y_test).long()

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, num_features, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out

# Set the hyperparameters
num_features = X_train.shape[1]
num_classes = 2
lr = 0.01
num_epochs = 10
batch_size = 32

# Initialize the model, loss function, and optimizer
model = LogisticRegression(num_features, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train the model
for epoch in range(num_epochs):
    for i in range(0, X_train.shape[0], batch_size):
        # Extract the batch
        inputs = X_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
    # Print the loss after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model on the test set
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = accuracy_score(y_test, predicted.numpy())
    f1 = f1_score(y_test, predicted.numpy(), average="macro")

# Print the evaluation metrics
##print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")

from sklearn.metrics import classification_report
print(classification_report(y_test, predicted.numpy()))


Epoch [1/10], Loss: 0.9475
Epoch [2/10], Loss: 0.6455
Epoch [3/10], Loss: 0.4267
Epoch [4/10], Loss: 0.3748
Epoch [5/10], Loss: 0.3505
Epoch [6/10], Loss: 0.3270
Epoch [7/10], Loss: 0.3037
Epoch [8/10], Loss: 0.2855
Epoch [9/10], Loss: 0.2730
Epoch [10/10], Loss: 0.2632
              precision    recall  f1-score   support

           0       0.84      0.75      0.79       201
           1       0.77      0.85      0.81       199

    accuracy                           0.80       400
   macro avg       0.81      0.80      0.80       400
weighted avg       0.81      0.80      0.80       400



In [3]:
## Second model:

## The GloVe embeddings glove.6B.100d to generate the input features. 


import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchtext.vocab import GloVe

# Load the GloVe embeddings
glove = GloVe(name='6B', dim=100)

# Load the data
data = pd.read_csv('aclImdb_v1_2000.csv')
reviews = data['review'].values
labels = data['sentiment'].values

# Split the data into train and test sets
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Convert the reviews to embeddings
def get_embedding(text):
    tokens = text.lower().split()
    embeddings = []
    for token in tokens:
        if token in glove.stoi:
            embeddings.append(glove.vectors[glove.stoi[token]])
    if embeddings:
        embeddings = np.stack(embeddings)
        embedding = embeddings.mean(axis=0)
    else:
        embedding = np.zeros((glove.dim,))
    return embedding

train_embeddings = np.array([get_embedding(text) for text in train_reviews])
test_embeddings = np.array([get_embedding(text) for text in test_reviews])

# Convert the data to PyTorch tensors
train_embeddings = torch.tensor(train_embeddings, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_embeddings = torch.tensor(test_embeddings, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out

# Initialize the model and the loss function
model = LogisticRegression(100, 2)
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters())

# Train the model
num_epochs = 10
batch_size = 64
total_steps = len(train_embeddings) // batch_size

for epoch in range(num_epochs):
    for i in range(total_steps):
        batch_embeddings = train_embeddings[i*batch_size:(i+1)*batch_size]
        batch_labels = train_labels[i*batch_size:(i+1)*batch_size]
        optimizer.zero_grad()
        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

# Evaluate the model
with torch.no_grad():
    test_outputs = model(test_embeddings)
    test_predictions = torch.argmax(test_outputs, dim=1)
    accuracy = accuracy_score(test_labels, test_predictions)
    f1 = f1_score(test_labels, test_predictions, average='weighted')

from sklearn.metrics import classification_report
print(classification_report(test_labels, test_predictions))







.vector_cache/glove.6B.zip: 862MB [02:40, 5.38MB/s]                           
100%|█████████▉| 399999/400000 [00:16<00:00, 24747.80it/s]


              precision    recall  f1-score   support

           0       0.67      0.68      0.67       201
           1       0.67      0.67      0.67       199

    accuracy                           0.67       400
   macro avg       0.67      0.67      0.67       400
weighted avg       0.67      0.67      0.67       400



In [ ]:
## Save the DataFrame to CSV

import os
import pandas as pd
import tarfile
import urllib.request

# Download the dataset
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = 'aclImdb_v1.tar.gz'
urllib.request.urlretrieve(url, filename)

# Extract the dataset
with tarfile.open(filename, 'r:gz') as tar:
    tar.extractall()

# Create a Pandas DataFrame from the dataset
rows = []
labels = {'pos': 1, 'neg': 0}
for split in ['train', 'test']:
    for label in ['pos', 'neg']:
        folder = f'aclImdb/{split}/{label}'
        for filename in os.listdir(folder):
            with open(os.path.join(folder, filename), 'r') as file:
                review = file.read()
            rows.append({'review': review, 'sentiment': labels[label]})
df = pd.DataFrame(rows)

# Save the DataFrame to CSV
df.to_csv('aclImdb_v1.csv', index=False)